## 40. 最小的k个数

输入n个整数，找出其中最小的K个数。例如输入4,5,1,6,2,7,3,8这8个数字，则最小的4个数字是1,2,3,4。

### 分析
如果想先把数组排序的话，最快的时间复杂度也是$O(n \log n)$。下面寻找一些复杂度为$O(n)$的算法。

[//]: # (<img src="images/img123.png" style="width: 400px;"/>)

### 解法一：基于Partition函数的时间复杂度为$O(n)$的算法
只有当我们可以修改输入的数组时才可用。

如果基于数组的第k个数字来调整，则使得比第k个数字小的所有数字都位于数组左边，比第k个数字大的所有数字都位于数组右边。

[关于partition函数的讲解](https://www.geeksforgeeks.org/quick-sort/)

In [3]:
def get_least_numbers(input, k):
    if check_invalid_array(input) or k > len(input):
        return []
    
    start = 0
    end = len(input) - 1
    index = partition(input, start, end)
    
    while index != k - 1:
        if index > k - 1:
            end = index - 1
            index = partition(input, start, end)
        else:
            start = index + 1
            index = partition(input, start, end)
    
    output = []
    for i in range(k):
        output.append(input[i])
    
    return output
    

def check_invalid_array(numbers):
    input_invalid = False
    if numbers is None or len(numbers) == 0:
        input_invalid = True
    return input_invalid

################################
# partition algorithm          #
################################
def partition(data, start, end):
    """
    This function takes last element as pivot, places
    the pivot element at its correct position in sorted
    array, and places all smaller (smaller than pivot)
    to left of pivot and all greater elements to right
    of pivot
    """
    if data is None or len(data) <= 0 or start < 0 or end >= len(data):
        raise Exception('Invalid Parameters')
    
    # pivot (Element to be placed at right position)
    pivot = data[end]
    i = start - 1 # index of smaller element
    
    for j in range(start, end):
        # if current element is smaller than or equal to pivot
        if data[j] <= pivot:
            i += 1 # increase the index of smaller element
            data[i], data[j] = data[j], data[i]
            
    data[i+1], data[end] = data[end], data[i+1]
    return i + 1
    

In [4]:
# Test
a = [4, 5, 6, 1, 2, 7, 3, 8]
print(get_least_numbers(a, 4))

[1, 2, 3, 4]


这个思路有一定的限制，因为partition函数会调整数组中数字的顺序。

###  解法二： 时间复杂度为$O(n \log k)$的算法，适合处理海量数据
1. 创建一个容量为k的容器，遍历input前k个数字时直接放进这个容器里。
2. 容器满了之后，将待插入的整数和容器内的最大值进行比较，决定进行替换或者不插入。

观察到容器满了之后，我们要做3件事情。

一是在k个整数中找到最大数；二是有可能在这个容器中删除最大数；三是有可能要插入一个新的数字。

如果用**二叉树**来实现这个容器，那么我们能在$O(\log k)$时间内实现这3步操作。因此对于n个输入数字而言，总的时间效率就是$O(n \log k)$

##### 如何选择二叉树的类型？
由于每次都需要找到k个整数中的最大数字，我们很容易想到用[max heap](http://courses.cs.vt.edu/cs2604/spring02/Notes/C07.Heaps.pdf). 在最大堆中，根节点的值总是大于它的子树中任意节点的值。于是我们每次可以在O(1)时间内得到已有的k个数字中的最大值，但是需要O(log k)时间完成删除已经插入操作。

Python里的`heapq`库已经有了完整的`min heap`和不是很完整的`max heap`的implementation。
```python
import heapq
############
# min heap #
############
minheap = [1, 2, 3, 4, 5, 6, 7, 8, 9]    
heapq.heapify(minheap)       
# Get current min
current_min = heapq.heappop(minheap) 
# Push new element in minheap
heapq.heappush(minheap, 10)

############
# max heap #
############
maxheap = [1, 2, 3, 4, 5, 6, 7, 8, 9] 
heapq._heapify_max(maxheap)
# Get current max
current_max = heapq._heappop_max(maxheap)
# Push new element in maxheap, but you need to rearrange it to a maxheap
heapq.heappush(minheap, 10)
heapq._heapify_max(maxheap)
```

In [5]:
import heapq

def get_least_numbers2(input, k):
    if check_invalid_array(input) or k > len(input):
        return []
    
    maxheap_container = []
    for i in range(len(input)):
        if len(maxheap_container) == k:
            # find current_max
            current_max = heapq._heappop_max(maxheap_container)
            # compare current_max with input[i]
            if input[i] < current_max:
                heapq.heappush(maxheap_container, input[i])
                heapq._heapify_max(maxheap_container)
            else:
                heapq.heappush(maxheap_container, current_max)
                heapq._heapify_max(maxheap_container)
        else:
            maxheap_container.append(input[i])
            heapq._heapify_max(maxheap_container)
    
    return maxheap_container

def check_invalid_array(numbers):
    input_invalid = False
    if numbers is None or len(numbers) == 0:
        input_invalid = True
    return input_invalid

### Test

In [6]:
# Test
a = [4, 5, 6, 1, 2, 7, 3, 8]
print(get_least_numbers2(a, 4))

[4, 3, 1, 2]


### 两种解法的特点比较
<img src="images/img40.png" style="width: 500px;"/>